<a href="https://colab.research.google.com/github/Qcritt/cosc470s24/blob/main/EclipseLab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
### Video
import cv2
import os


def extract_frames(video_path, output_dir):
    # Make sure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    vidcap = cv2.VideoCapture(video_path)

    success, image = vidcap.read()
    count = 0

    while success:
        # Save frame as JPEG file
        cv2.imwrite(os.path.join(output_dir, f"frame{count:04d}.jpg"), image)

        success, image = vidcap.read()
        print(f'Read a new frame: {success}', end='\r')
        count += 1

# Example usage
video_path = '/Users/quincycrittendon/Desktop/ResizedFrames/AdobeStock_191321618_Video_HD_Preview (1).mov'
output_dir = '/Users/quincycrittendon/Desktop/ResizedFrames'

extract_frames(video_path, output_dir)

In [10]:
### Resize
import os
from PIL import Image

def resize_images(directory):
    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(directory) if f.endswith('.jpg') or f.endswith('.png')]

    for file in image_files:
        # Open the image file
        image = Image.open(os.path.join(directory, file))

        # Calculate the dimensions for cropping
        width, height = image.size
        if width > height:
            left = (width - height) // 2
            right = left + height
            top = 0
            bottom = height
        else:
            top = (height - width) // 2
            bottom = top + width
            left = 0
            right = width

        # Crop the image to the middle
        image = image.crop((left, top, right, bottom))

        # Resize the image to 32x32 pixels
        image = image.resize((32, 32))

        # Save the resized image
        image.save(os.path.join(directory, f'resized_{file}'))

    print('Image resizing complete.')

# Specify the directory containing the images
directory = '/Users/quincycrittendon/Desktop/ResizedFrames'

# Call the resize_images function
resize_images(directory)

Image resizing complete.


In [7]:

import os
from PIL import Image
import numpy as np

def load_custom_images(directory, size=(32,32)):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the image
        path = os.path.join(directory, filename)
        # Open the image file
        image = Image.open(path)
        # Resize image (just in case some are not 32x32)
        image = image.resize(size)
        # Convert image to numpy array
        image_data = np.asarray(image)
        # Scale data to the range [-1, 1]
        image_data = (image_data - 127.5) / 127.5
        images.append(image_data)
    # Convert list to a numpy array
    images = np.array(images)
    return images

In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images_as_dataset(directory, size=(32,32), batch_size=32, shuffle_buffer_size=1000):
    global total_num_images
    global images
    images = []
    for filename in os.listdir(directory):
        if filename.startswith("resized") and filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=size)
            img = img_to_array(img)
            img = (img - 127.5) / 127.5  # Normalize the images to [-1, 1]
            images.append(img)
    total_num_images = len(images)
    images = np.array(images)
    dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(shuffle_buffer_size).batch(batch_size, drop_remainder=True)
    return dataset



# Directory containing your images
directory ='/Users/quincycrittendon/Desktop/ResizedFrames'
batch_size = 32
total_num_images = 0 # will be updated by load func
dataset = load_images_as_dataset(directory, batch_size=batch_size)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU

def make_generator_model():
    model = Sequential([
        Dense(8*8*256, use_bias=False, input_shape=(100,)),
        BatchNormalization(),
        LeakyReLU(),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3]),
        LeakyReLU(),
        Flatten(),
        Dense(1)
    ])
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

import tensorflow as tf

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def train_step(images):
    print("Shape of images before reshaping:", images.shape)  # Debug statement

    # Ensure the input has the correct shape
    images = tf.reshape(images, (-1, 32, 32, 3))  # -1 is used to automatically calculate the needed batch size

    print("Shape of images after reshaping:", images.shape)


def train_step(images):
    # Ensure the input has the correct shape
    images = tf.reshape(images, (-1, 32, 32, 3))  # -1 is used to automatically calculate the needed batch size

    noise = tf.random.normal([len(images), 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, epochs, batch_size):
    global total_num_images
    for epoch in range(epochs):
        print(f"Epoch {epoch} starting ...")
        for batchi, image_batch in enumerate(dataset):
            if batchi % 10 == 0:
                print(f"Batch {batchi}...")
            train_step(image_batch)
        print("complete!")
        dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(tf.data.experimental.UNKNOWN_CARDINALITY).batch(batch_size, drop_remainder=True)
        if epoch % 5 == 0:
            show_gan(10, epoch)


def show_gan(num_images, cnt):
    # Generate images from the noise vector
    noise = tf.random.normal([num_images, 100])
    generated_images = generator(noise, training=False)

    # Adjusting the pixel values to display them properly
    generated_images = (generated_images + 1) / 2  # rescale from [-1, 1] to [0, 1]
    generated_images = generated_images.numpy()  # convert to numpy array if not already

    # Create a plot to display the images
    fig, axes = plt.subplots(1, 10, figsize=(20, 2))
    for i, img in enumerate(generated_images):
        axes[i].imshow(img)
        axes[i].axis('off')  # Turn off axis labels
    fig.savefig(f"generated{cnt}.png", dpi=300)
    plt.close(fig)


train(dataset, 200, batch_size)  # Train for 200 epochs

Epoch 0 starting ...
complete!
Epoch 1 starting ...
complete!
Epoch 2 starting ...
complete!
Epoch 3 starting ...
complete!
Epoch 4 starting ...
complete!
Epoch 5 starting ...
complete!
Epoch 6 starting ...
complete!
Epoch 7 starting ...
complete!
Epoch 8 starting ...
complete!
Epoch 9 starting ...
complete!
Epoch 10 starting ...
complete!
Epoch 11 starting ...
complete!
Epoch 12 starting ...
complete!
Epoch 13 starting ...
complete!
Epoch 14 starting ...
complete!
Epoch 15 starting ...
complete!
Epoch 16 starting ...
complete!
Epoch 17 starting ...
complete!
Epoch 18 starting ...
complete!
Epoch 19 starting ...
complete!
Epoch 20 starting ...
complete!
Epoch 21 starting ...
complete!
Epoch 22 starting ...
complete!
Epoch 23 starting ...
complete!
Epoch 24 starting ...
complete!
Epoch 25 starting ...
complete!
Epoch 26 starting ...
complete!
Epoch 27 starting ...
complete!
Epoch 28 starting ...
complete!
Epoch 29 starting ...
complete!
Epoch 30 starting ...
complete!
Epoch 31 starting 

In [21]:
###pre process
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.startswith("resized") and (filename.endswith(".jpg") or filename.endswith(".png")):
            image_path = os.path.join(directory, filename)
            image = Image.open(image_path)
            #image = image.resize((64, 64))  # Resize the image to desired dimensions - no need to resize since we already have separate script that did that
            image = img_to_array(image)
            images.append(image)
    return np.array(images)

# Specify the directory containing the images
image_directory = "/Users/quincycrittendon/Desktop/ResizedFrames"

# Load images from the directory
dataset = load_images(image_directory)

# Normalize the pixel values to the range [-1, 1]
dataset = (dataset.astype(np.float32) - 127.5) / 127.5

# Print the shape of the dataset
print("Dataset shape:", dataset.shape)

Dataset shape: (0,)


In [1]:
import os
from PIL import Image
import numpy as np

def load_custom_images(directory, size=(32,32)):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the image
        path = os.path.join(directory, filename)
        # Open the image file
        image = Image.open(path)
        # Resize image (just in case some are not 32x32)
        image = image.resize(size)
        # Convert image to numpy array
        image_data = np.asarray(image)
        # Scale data to the range [-1, 1]
        image_data = (image_data - 127.5) / 127.5
        images.append(image_data)
    # Convert list to a numpy array
    images = np.array(images)
    return images


In [23]:
# Load your custom dataset
dataset = load_custom_images('/Users/quincycrittendon/Desktop/ResizedFrames')

# Verify the shape and data type
print('Dataset shape:', dataset.shape)
print('Data type:', dataset.dtype)


Dataset shape: (0,)
Data type: float64


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout
from keras.optimizers import Adam
import numpy as np

def define_discriminator(in_shape=(32, 32, 3)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
    return model


In [25]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [36]:
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
import numpy as np

def train_gan(g_model, d_model, gan_model, dataset, latent_dim, epochs=2, batch_size=128):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        for batch in range(dataset.shape[0] // batch_size):
            # randomly select real samples
            idx = np.random.randint(0, dataset.shape[0], half_batch)
            real_images = dataset[idx]
            real_y = np.ones((half_batch, 1))
            # generate fake images
            noise = np.random.randn(latent_dim * half_batch).reshape(half_batch, latent_dim)
            fake_images = g_model.predict(noise)
            fake_y = np.zeros((half_batch, 1))
            # update discriminator model
            d_model.train_on_batch(real_images, real_y)
            d_model.train_on_batch(fake_images, fake_y)
            # prepare points in latent space as input for the generator
            x_gan = np.random.randn(latent_dim * batch_size).reshape(batch_size, latent_dim)
            y_gan = np.ones((batch_size, 1))
            # update the generator via the discriminator's error
            gan_model.train_on_batch(x_gan, y_gan)

            # Plot a generated image after every batch
            if batch % 1 == 0:
                show_generated_image(g_model, latent_dim)

def show_generated_image(g_model, latent_dim):
    noise = np.random.randn(latent_dim).reshape(1, latent_dim)
    image = g_model.predict(noise)[0]
    image = (image + 1) / 2.0  # Rescale to [0, 1]
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [ ]:
# Parameters
latent_dim = 100  # Dimensionality of the latent space

# Define models
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)

# Load dataset (assuming this is already done)
# dataset = load_custom_images('path/to/EclipseFrames')

# Train the GAN
train_gan(generator, discriminator, gan_model, dataset, latent_dim, epochs=100, batch_size=128)
